In [102]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder

from feature_engineering import cluster_two_features
from data_loader import load_data, get_entire_data, get_features

In [3]:
IS_CUSTOM = False


In [4]:
df["KnowledgeTag_first3_clust"] = cluster_two_features(df, "KnowledgeTag", "first3", 44)

In [105]:
def get_binning_data(df, col, n_bins=10, encode='ordinal', strategy='quantile'):
    """
    #1. strategy = 'uniform'
    #2. strategy = 'quantile'
    #3. strategy = 'kmeans'
    """
    train_pt = pd.DataFrame(df[col])
    est_uni = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy=strategy)
    est_uni.fit(train_pt)
    Xt_uni=est_uni.transform(train_pt)
    print("edges : ", est_uni.bin_edges_[0])
    unique, counts = np.unique(Xt_uni, return_counts=True)
    print(dict(zip(unique, counts)))
    return Xt_uni.squeeze(1)
    
def category_data_loader(IS_CUSTOM=False, n_bins=10, encode='ordinal', strategy='quantile'):
    df = get_features(get_entire_data(*load_data(IS_CUSTOM=IS_CUSTOM)))
    cat_features = []
    bin_list = []
    new_df = df.copy()
    for col in df.columns:
        if (df[col].dtype == int) or (df[col].dtype == float and df[col].nunique() < 100):
            new_df[col] = df[col].fillna(df[col].mean())
            le = LabelEncoder()
            new_df[col] = le.fit_transform(new_df[col])
            # new_df[col] = ((df[col]-df[col].min())*df[col].nunique()).astype(int)
            cat_features.append(col)
        elif df[col].dtype==float:
            bin_list.append(col)
            df[col] = df[col].fillna(df[col].mean())
            cat_features.append(col)
            continue
        else:
            le = LabelEncoder()
            new_df[col] = le.fit_transform(new_df[col])
            # new_df[col] = ((df[col]-df[col].min())*df[col].nunique()).astype(int)
            cat_features.append(col)
    print("Start binning")
    for col in bin_list:
        new_df[col] = get_binning_data(new_df, col, n_bins=int(df[col].nunique()**0.5), strategy="kmeans")
    return cat_features, new_df
cat_ft, cat_df = category_data_loader(df)

Start binning
edges :  [0.         0.29375957 0.38729214 0.45945015 0.52782488 0.59408683
 0.65469775 0.71299719 0.7705678  0.83053353 1.        ]
{0.0: 41260, 1.0: 112923, 2.0: 138989, 3.0: 195161, 4.0: 261009, 5.0: 345387, 6.0: 388991, 7.0: 397400, 8.0: 370390, 9.0: 225196}
edges :  [0.         0.10796669 0.13434466 0.1580201  0.18044004 0.20198395
 0.2217679  0.23896951 0.25592304 0.29742336 0.4       ]
{0.0: 56073, 1.0: 117372, 2.0: 205253, 3.0: 266565, 4.0: 330280, 5.0: 367686, 6.0: 462460, 7.0: 651086, 8.0: 17986, 9.0: 1945}
edges :  [0.33017677 0.39942603 0.456388   0.51599222 0.57996428 0.64292884
 0.70306271 0.75851664 0.81205453 0.86732513 0.95503356]
{0.0: 63949, 1.0: 198896, 2.0: 270156, 3.0: 291466, 4.0: 301090, 5.0: 309134, 6.0: 338529, 7.0: 320994, 8.0: 249480, 9.0: 133012}
edges :  [0.04431649 0.06865402 0.09420694 0.11680392 0.13928644 0.16172027
 0.18415535 0.20615125 0.22553335 0.2409232  0.27117057]
{0.0: 14311, 1.0: 38814, 2.0: 81612, 3.0: 130257, 4.0: 190004, 5.0:

In [106]:
cat_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,userID_answerCode_mean,userID_answerCode_count,userID_answerCode_sum,userID_answerCode_var,...,hour_answerCode_mean,hour_answerCode_count,hour_answerCode_sum,hour_answerCode_var,hour_answerCode_Level,month_answerCode_mean,month_answerCode_count,month_answerCode_sum,month_answerCode_var,KnowledgeTag_first3_clust
0,0,5354,975,2,247401,556,5.0,730,470,6.0,...,14,11,11,9,2,9,4,6,2,22
1,0,5355,975,2,247402,557,5.0,730,470,6.0,...,14,11,11,9,2,9,4,6,2,22
2,0,5356,975,2,247404,557,5.0,730,470,6.0,...,14,11,11,9,2,9,4,6,2,22
3,0,5357,975,2,247406,557,5.0,730,470,6.0,...,14,11,11,9,2,9,4,6,2,22
4,0,5358,975,2,247408,557,5.0,730,470,6.0,...,14,11,11,9,2,9,4,6,2,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476701,7441,2373,456,1,703871,58,4.0,0,5,8.0,...,7,23,20,16,0,4,7,7,7,15
2476702,7441,3909,748,2,1358827,720,4.0,0,5,8.0,...,13,13,13,10,1,5,9,9,6,32
2476703,7441,3910,748,2,1358832,720,4.0,0,5,8.0,...,13,13,13,10,1,5,9,9,6,32
2476704,7441,3911,748,2,1358847,720,4.0,0,5,8.0,...,13,13,13,10,1,5,9,9,6,32
